In [1]:
proj_path = '/home/ajhnam/projects/hidden_singles_public/'

In [2]:
import sys
sys.path.append(proj_path + 'python/')

import random
import numpy as np
import itertools
import pandas as pd
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader as DataLoader
from tqdm.auto import tqdm


from hiddensingles.misc import torch_utils as tu
from hiddensingles.misc import utils, TensorDict, TensorDictDataset, RRN, DigitRRN
from hiddensingles.experiment.sudoku_hs_service import create_tutorial, create_phase1, create_phase2

In [3]:
device = 1

In [4]:
def get_results(model, dataset, num_steps=8):
    outputs = model(dataset.inputs, num_steps=num_steps)
    
    goals = tu.expand_along_dim(dataset.goals, 1, num_steps)
    goal_outputs = tu.select(outputs, goals, select_dims=1)
    
    targets = tu.expand_along_dim(dataset.targets, 1, num_steps)
    goal_loss = tu.cross_entropy(goal_outputs, targets)
    goal_probs = tu.select(goal_outputs.softmax(-1), dataset.targets)
    goal_td = TensorDict(loss=goal_loss,
                         probs=goal_probs,
                         outputs=goal_outputs)
    
    coords = tu.expand_along_dim(dataset.coords, 1, num_steps)
    out_exp = tu.expand_along_dim(outputs, 2, 9)
    coord_outputs = tu.select(out_exp, coords, select_dims=1)
    coord_targets = tu.expand_along_dim(dataset.coord_targets, 1, num_steps)
    coord_loss = tu.cross_entropy(coord_outputs, coord_targets)
    coord_probs = tu.select_subtensors(coord_outputs.softmax(-1), coord_targets)
    coord_td = TensorDict(loss=coord_loss,
                          probs=coord_probs,
                          outputs=coord_outputs)

    loss = goal_loss + coord_loss
    return TensorDict(loss=loss,
                      outputs=outputs,
                      goal=goal_td,
                      coord=coord_td)

In [5]:
def get_phase2_conditions(phase2):
    ht = [p.condition.house_type for p in phase2]
    hi = [p.condition.house_index for p in phase2]
    ci = [p.condition.cell_index for p in phase2]
    ds = [p.condition.digit_set for p in phase2]
    conditions = pd.DataFrame(np.array([ht, hi, ci, ds]).T,
                              columns=['house_type', 'house_index', 'cell_index', 'digit_set'])
    return conditions

def hidden_singles_to_tensordict(list_of_hidden_singles, digit_rrn):
    grids = torch.tensor([a.grid.array for a in list_of_hidden_singles], device=device)
    goals = [p.coordinates['goal'] for p in list_of_hidden_singles]
    goals = torch.tensor([[g.x, g.y] for g in goals], device=device)
    targets = torch.tensor([a.digits['target'] for a in list_of_hidden_singles], device=device) - 1 # make it 0-8
    coords = grids.nonzero()[:,1:].view(len(list_of_hidden_singles), -1, 2)
    coord_targets = tu.select(tu.expand_along_dim(grids, 1, 9), coords) - 1 # make it 0-8
    
    inputs = DigitRRN.make_onehot(grids) if digit_rrn else grids
    return TensorDict(inputs=inputs,
                      grids=grids,
                      goals=goals,
                      targets=targets,
                      coords=coords,
                      coord_targets=coord_targets)

def create_dataset(num_train, num_valid, digit_rrn=False):
    digit_set1 = set(random.sample(set(range(1, 10)), 4))
    digit_set2 = set(random.sample(set(range(1, 10)) - digit_set1, 4))
    tutorial = create_tutorial(digit_set1)
    phase1 = create_phase1(tutorial, num_train + num_valid)
    phase2 = create_phase2(tutorial, digit_set1, digit_set2)
    conditions = get_phase2_conditions(phase2)

    phase1 = hidden_singles_to_tensordict(phase1, digit_rrn=digit_rrn)
    phase2 = hidden_singles_to_tensordict([p.hidden_single for p in phase2], digit_rrn=digit_rrn)
    
    dataset = TensorDict(train=phase1[:num_train],
                         valid=phase1[num_train:],
                         test=phase2)
    return dataset, conditions

def train_model(model, dataset, num_steps=8,
                batch_size=100, num_epochs=100, lr=1e-3,
                record_epoch=1, show_pbar=True, verbose=False):
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    dataloader = DataLoader(TensorDictDataset(dataset.train), batch_size=batch_size, shuffle=True)
    
    iterator = range(num_epochs + 1)
    if show_pbar:
        iterator = tqdm(iterator, leave=False)

    all_results = []
    
    for i in iterator:
        epoch_results = []
        
        if i%record_epoch == 0:
            with torch.no_grad():
                v_results = get_results(model, dataset.valid)
            v_predictions = v_results.goal.outputs[:,-1].argmax(-1)
            v_accuracy = (v_predictions == dataset.valid.targets).float().mean().item()
            v_probability = v_results.goal.probs[:,-1].mean().item()
            
        goal_loss = []
        for dset in dataloader:
            dset = TensorDict(**dset)
            optimizer.zero_grad()
            results = get_results(model, dset, num_steps=num_steps)
            
            goal_loss.append(results.goal.loss.item())
            results.loss.backward()
            optimizer.step()
            
            if i%record_epoch == 0:
                predictions = results.goal.outputs[:,-1].argmax(-1)
                correct = (predictions == dset.targets)
                results.goal.correct = correct
                epoch_results.append(results.goal[['correct', 'probs']].detach())
            
        if i%record_epoch == 0:
            epoch_results = TensorDict.cat(epoch_results, dim=0)
            tr_accuracy = epoch_results.correct.float().mean().item()
            tr_probability = epoch_results.probs[:,-1].mean().item()
            
            row = {'epoch': i,
                   'loss': np.mean(goal_loss),
                   'tr_probability': tr_probability,
                   'tr_accuracy': tr_accuracy,
                   'v_probability': v_probability,
                   'v_accuracy': v_accuracy}
            all_results.append(row)
            
            if verbose:
                utils.kv_print(**row)
                
    results = pd.DataFrame(all_results)
    return results

def get_test_results(model, dataset, conditions):
    with torch.no_grad():
        results = get_results(model, dataset.test)
        
    goal_probs = results.goal.probs[:,-1].cpu().numpy()
    predictions = results.goal.outputs[:,-1].argmax(-1)
    correct = (predictions == dataset.test.targets).cpu().numpy()
    
    test_results = conditions.copy()
    test_results['probability'] = goal_probs
    test_results['correct'] = correct
    return test_results

# Training and Test Results

In [17]:
all_train_results = []
all_test_results = []
for num_train in tqdm([25, 50, 100, 200, 300, 400, 500]):
    model = RRN(digit_embed_size=10,
                num_mlp_layers=0,
                hidden_vector_size=48,
                message_size=48,
                encode_coordinates=False).to(device)
    dataset, conditions = create_dataset(num_train=num_train, num_valid=100, digit_rrn=False)
    train_results = train_model(model, dataset, num_steps=8, batch_size=100,
                                num_epochs=1000, record_epoch=10, verbose=False)
    test_results = get_test_results(model, dataset, conditions)
    train_results['train_size'] = num_train
    test_results['train_size'] = num_train
    
    all_train_results.append(train_results)
    all_test_results.append(test_results)
    
train_results = pd.concat(all_train_results)
test_results = pd.concat(all_test_results)

train_results = train_results[['train_size', 'epoch', 'loss',
                               'tr_accuracy', 'tr_probability', 'v_accuracy', 'v_probability']]
test_results = test_results[['train_size', 'house_type', 'house_index', 'cell_index',
                             'digit_set', 'correct', 'probability']]

In [18]:
# Save results

dirpath = proj_path + 'data/rrn/'
utils.mkdir(dirpath)

train_results.to_csv(dirpath + "rrn_train_results.tsv", sep='\t', index=False)
test_results.to_csv(dirpath + "rrn_test_results.tsv", sep='\t', index=False)

# Digit RRN

In [21]:
all_train_results = []
all_test_results = []
for num_train in tqdm([25, 50, 100]):
    model = DigitRRN(hidden_vector_size=8,
                     message_size=8).to(device)
    dataset, conditions = create_dataset(num_train=num_train, num_valid=100, digit_rrn=True)
    train_results = train_model(model, dataset, num_steps=8, batch_size=20,
                                num_epochs=500, record_epoch=1, verbose=False)
    test_results = get_test_results(model, dataset, conditions)
    train_results['train_size'] = num_train
    test_results['train_size'] = num_train
    
    all_train_results.append(train_results)
    all_test_results.append(test_results)
    
train_results = pd.concat(all_train_results)
test_results = pd.concat(all_test_results)

train_results = train_results[['train_size', 'epoch', 'loss',
                               'tr_accuracy', 'tr_probability', 'v_accuracy', 'v_probability']]
test_results = test_results[['train_size', 'house_type', 'house_index', 'cell_index',
                             'digit_set', 'correct', 'probability']]

In [22]:
# Save results

dirpath = proj_path + 'data/rrn/'
utils.mkdir(dirpath)

train_results.to_csv(dirpath + "drrn_train_results.tsv", sep='\t', index=False)
test_results.to_csv(dirpath + "drrn_test_results.tsv", sep='\t', index=False)